<h1> TF-Serving: BERT</h1>
- Windows mit WLS2 -

Mit TF-Serving kann problemlos ein Server erstellt und konfiguriert werden, der das Model mit Anfragen beliefert und als Rückgabe Predictions ausgibt.

Anders als bei Flask und Fast-API, bietet TF-Serving Features, die den Umgang vereinfachen.
- Es können Batch-Predictions ausgeführt werden.
- Timeouts können eingestellt werden.
- Verschiedene Versionen (management von versionen) und Konfigurationen.
- Durch Docker einfaches Monitoring, einfaches Starten und Stoppen des Servers. Container kann beliebig konfiguriert und erweitert werden.
- Zuweisung von Ressourcen.
- ...

Auch andere Arten von Modellen können verwendet werden.

Weitere inhaltliche Details sind auf der Tensorflow Seite zu finden.
> https://www.tensorflow.org/tfx/serving/docker [Letzter Zugriff: 21.06.2024] <br>
> https://www.tensorflow.org/tfx/serving/architecture [Letzter Zugriff: 21.06.2024] <br>
> https://www.tensorflow.org/tfx/serving/serving_config [Letzter Zugriff: 21.06.2024]

<i>Abb1</i>: Überblick TF-Serving


<img src="./data/img/tf_serving_1.PNG" width=600 hight=500 > 

Für das Model nehmen wir das Trainierte BERT Model für die Klassifizierung von Filmbewertungen.

<h2>Basis Set-Up</h2>

Als Erstes wird das Model gespeichert, hier wird es in einem Windowsverzeichnis abgelegt. 

Model Version 1 wird in dem Verzeichnis mit dem Namen 1 abgelegt.

Unter Windows: <br>
`model.save('C:/.../tf_serving/movie_review/1', include_optimizer=False)`

Unter WLS: <br>
Dann wird der TF-Serving Container heruntergeladen, wie auf der Webseite beschrieben.:<br>
`# Download the TensorFlow Serving Docker image and repo ` <br>
`docker pull tensorflow/serving`

Danach sollte der Container sichtbar sein.

Mit dem Befehl `docker images` werden alle Container gelistet.

<i>Abb2</i>: Liste Container in Docker


<img src="./data/img/tf_serving_2.PNG" width=600 hight=500 > 

Der Container beinhaltet alles was nötig ist, um TF-Serving zu nutze.<br>
Der Container kann, wenn nötig, weiter modifiziert werden.

Für den `docker run` Befehl gibt es zahlreiche Parameter, die genutzt werden können.

<b>Starte den Container:</b>

<b>--entrypoint</b>: Einstieg in die Konsole des Containers. Hier können wir dann manuell den Server starten. <br>
<b>-v /mnt/c/.../tf_serving:/tf_serving</b>: Mit `/mnt` kann auf die Datenträger zugegriffen werden z. B. C, von dort aus kann das Workspace mitgegeben werden. <br>
Dabei ist das Verzeichnis tf_serving:  `.../tf_serving/movie_review/1/<-saved_model->`

Der Name des Models wird auch mitgegeben. 

Wenn der Container läuft, können weitere Befehle gegeben werden. <br>
Um TF-Serving zu startet, wird ein bestimmter Befehl ausgeführt:

<b>--model_base_path</b>: Der Pfad zum Modelverzeichnis, wo sich die verschiedenen Versionen befinden.

Danach sollte der Server starten. Die Abbildung 3 zeigt die Ausgabe in der Konsole (im Docker Container)

<i>Abb3</i>: TF-Serving gestartet.


<img src="./data/img/tf_serving_3.PNG" width=700 hight=400 > 

<h2>Abfrage mit Curl</h2>

Um es zu testen, kann eine zweite Konsole geöffnet werden, um eine Request zu senden. 

`curl -d '{"instances": ["The movie was not good, it was bad"]}'  -X POST http://localhost:8501/v1/models/movie_review:predict`

Dabei soll `"The movie was not good, it was bad"` als negatives Review klassifiziert werden.

<i>Abb4</i>: Antwort des Servers.


<img src="./data/img/tf_serving_4.PNG" width=500 hight=200 > 

Diese Antwort bedeutet, dass dieses Review negativ war.

<h2>Abfrage mit Json</h2>

Im Alltag würde man nicht die Konsole für die Abfragen nutzen. 
- Ist aber gut, um schnell kleine Testabfragen durchzuführen. 

In Anwendung z. B. unter Streamlit, würde man diese Abfrage in einer Funktion unterbringen, wo ggf. noch Einstellungen vorgenommen werden.

In [1]:
import json
import requests
model_name:str= "movie_review"
url = f'http://localhost:8501/v1/models/{model_name}:predict'

In [2]:
# Funktion für die Abfrage.
def make_prediction(instances:list):
   data = json.dumps({"signature_name": "serving_default", "instances": instances})
   headers = {"content-type": "application/json"}
   json_response = requests.post(url, data=data, headers=headers)
   predictions = json.loads(json_response.text)['predictions']
   return predictions

<u>Hinweis</u>:<br>
Das Model hat ein Layer, der das Vorverarbeiten der Texte übernimmt. <br>
Sonst könnte man den Text nicht einfach so als Request schicken.

In [7]:
# Beispielhafte Reviews. 
# - 1 Positives und negatives Review. 
reviews=[
    "This movie was really good, I loved the part where the hero saved the world, I recommend you all to watch it!!!!!",   
    "Bruh, this movie was trash, I don't understand why poeple liked it. Boring story and the effects were cheesy. To make it short, boring, trash,  24/7 movie, medicore. Not gonna watch this again xd"                                 
]

In [8]:
# Stelle Anfrage. 
make_prediction(reviews)

[[0.748233378], [0.313450456]]

<h1>Model Versionen und Konfigurationsdatei</h1>

Um es übersichtlich und verständlich zu halten, wird das Model in dem Verzeichnis einfach zweimal kopiert- jetzt haben wir drei „verschiedene“ Versionen eines Models.

`C:/.../tf_serving/my_models/movie_review/1`<br>
`C:/.../tf_serving/my_models/movie_review/2`<br>
`C:/.../tf_serving/my_models/movie_review/3`<br>

und: <br>

`C:/.../tf_serving/my_models/movie_review_2/1`<br>
`C:/.../tf_serving/my_models/movie_review_2/2`<br>
`C:/.../tf_serving/my_models/movie_review_2/3`<br>

<h2>Wähle alle Versionen</h2>

Jetzt legen wir eine Konfigurationsdatei an im Verzeichnis `tf_serving/`

Als erstes: <br>
Wir nutzen nur das Model <b>movie_review</b> mit allen Versionen.
- Achte dabei auf die richtige Angabe der Pfade.

<b>--model_config_file_poll_wait_seconds</b>: Schaue alle n-Sekunden nach, ob sich die Konfigurationsdatei geändert hat. <br>
<b>--model_config_file=</b>: Gebe den Pfad der Datei an, hier: `/tf_serving/model_config.a`

Docker Container Starten: <br>
`tensorflow_model_server --port=8500 --rest_api_port=8501 --model_config_file_poll_wait_seconds=10 --model_config_file=/tf_serving/model_config.a`

<i>Abb5</i>: Auszug der Konsolenausgabe: alle drei Modelle geladen.


<img src="./data/img/tf_serving_5.PNG" width=800 hight=500 > 

<h2>Wähle bestimme Versionen eines Models</h2>

Bei der Angabe verschiedener Versionen sieht die Datei so aus: 

Docker Container Starten: <br>
`tensorflow_model_server --port=8500 --rest_api_port=8501 --model_config_file_poll_wait_seconds=60 --model_config_file=/tf_serving/model_config.a --allow_version_labels_for_unavailable_models=True`

<u>Hinweis</u>: <br>
Hier muss der Parameter `--allow_version_labels_for_unavailable_models=True` mitgegeben werden. <br>
Labels können nur bei geladenen Modellen angebracht werden. <br>
Am Ende werden die spezifischen Modelle geladen und mit einem Label versehen. 

Siehe: https://www.tensorflow.org/tfx/serving/serving_config [Letzter Zugriff: 21.06.2024]

<h2>Zwei Modelle mit verschiedenen Versionen</h2>

Mit der Angabe eines zweiten Models und deren Versionen sieht das ganze dann so aus:

Docker Container Starten: <br>
`tensorflow_model_server --port=8500 --rest_api_port=8501 --model_config_file_poll_wait_seconds=60 --model_config_file=/tf_serving/model_config.a --allow_version_labels_for_unavailable_models=True`
